In [6]:
import psycopg2
import yaml
import stanza

import pandas as pd
import re
from bs4 import BeautifulSoup
import time

# Load configuration from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Database connection parameters
db_config = config['local_db']  # Adjust to 'remote_db' if needed

# Connect to the database
conn = psycopg2.connect(
    host=db_config['host'],
    port=db_config['port'],
    dbname=db_config['database'],
    user=db_config['user'],
    password=db_config['password']
)

# Define query parameters
query_limit = config.get('query_stanza_limit', 2500000)
offset = config.get('offset', 0)
query_text = 'gold mine'

# SQL query to retrieve stories containing the keyword "gold mine"
story_query = f"SELECT id, story FROM public.\"DJ_NEWS_STORIES\" WHERE story LIKE '%{query_text}%' LIMIT {query_limit} OFFSET {offset};"


# Load Stanza's English model for NER
stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,ner')

# Function to clean text: removes URLs, HTML tags, punctuation, numbers, and extra whitespace
def clean_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove punctuation, digits, and non-printable characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace

    return text

# Start processing
start_time = time.time()

# Execute query to retrieve stories
cursor = conn.cursor()
cursor.execute(story_query)
result = cursor.fetchall()

# Collect entities and counts
unique_entities_set = set()
all_entities = set()
type_counts = {}
total_count = 0

# Process each story in the results
for row in result:
    story_id = row[0]
    text = row[1]
    
    # Clean the story text
    cleaned_text = clean_text(text)
    
    # Process cleaned text with Stanza NER
    doc = nlp(cleaned_text)
    
    # Extract and store relevant entities
    for ent in doc.ents:
        if ent.type in ['ORG', 'GPE', 'PERSON']:  # Use relevant entity types
            entity_tuple = (ent.text.strip(), ent.type, story_id)
            all_entities.add(entity_tuple)
            unique_entities_set.add((ent.text.strip(), ent.type))
            
            # Update counts for summary
            type_counts[ent.type] = type_counts.get(ent.type, 0) + 1
            total_count += 1

# Convert sets to sorted DataFrames for saving to Excel
unique_entities_df = pd.DataFrame(sorted(unique_entities_set), columns=["Entity", "Tag"])
all_entities_df = pd.DataFrame(sorted(all_entities), columns=["Entity", "Tag", "Story ID"])
summary_df = pd.DataFrame(list(type_counts.items()), columns=["Tag", "Count"])
summary_df = pd.concat([summary_df, pd.DataFrame([["TOTAL", total_count]], columns=["Tag", "Count"])])

# Save to Excel with multiple sheets
with pd.ExcelWriter("recognized_entities_stanza.xlsx", engine="openpyxl") as writer:
    unique_entities_df.to_excel(writer, sheet_name="Unique Entities", index=False)
    all_entities_df.to_excel(writer, sheet_name="All Entities with Story ID", index=False)
    summary_df.to_excel(writer, sheet_name="Summary", index=False)

# Close the database connection
cursor.close()
conn.close()

# Print execution time
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")


2024-10-27 01:01:01 INFO: Downloading default packages for language: en (English)...
2024-10-27 01:01:01 INFO: File exists: /home/swiftx/stanza_resources/en/default.zip.
2024-10-27 01:01:04 INFO: Finished downloading models and saved to /home/swiftx/stanza_resources.
2024-10-27 01:01:04 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2024-10-27 01:01:04 INFO: Use device: cpu
2024-10-27 01:01:04 INFO: Loading: tokenize
2024-10-27 01:01:04 INFO: Loading: ner
2024-10-27 01:01:05 INFO: Done loading processors!


Execution time: 22222.26 seconds
